# 🖥️ Nested Grids

In some applications, you may have access to fields on different grids that each cover only part of the region of interest. Then, you would like to combine them all together. You may also have a grid covering the entire region and another one only covering part of it, but with a higher resolution. The set of those grids form what we call nested grids.

In Parcels v4, we can use the new `uxarray` integration to determine in which grid a particle is located. We will demonstrate how to set up a simulation with multiple nested grids, and how to handle particle transitions between these grids.


This tutorial shows how to use these Nested Fields with a very idealised example.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.tri as mtri
import numpy as np
import uxarray as ux
import xarray as xr
from matplotlib.colors import ListedColormap
from shapely.geometry import MultiPoint, Point, Polygon
from triangle import triangulate

import parcels
from parcels._datasets.structured.generated import simple_UV_dataset

## Setting up the individual Grids and Fields

First we define a helper function to quickly set up a nested dataset

In [ ]:
ds_in = []
grid_polygons = []


def setup_nested_ds(polygon):
    xdim, ydim = 20, 20
    ds = simple_UV_dataset(dims=(1, 1, ydim, xdim), mesh="spherical")
    ds["lon"][:] = np.linspace(polygon[:, 0].min(), polygon[:, 0].max(), xdim)
    ds["lat"][:] = np.linspace(polygon[:, 1].min(), polygon[:, 1].max(), ydim)
    return ds

Now we create a zonal and meridional velocity field defined on a small grid. Both the zonal (1m/s) and meridional (0 m/s) velocity are uniform.

In [ ]:
polygon = np.array([(10, 15), (25, 10), (25, 25), (10, 35)])
grid_polygons.append(polygon)

ds_in.append(setup_nested_ds(polygon))
ds_in[-1]["U"][:] = 1.0
ds_in[-1]["V"][:] = 0.0

Then, we define another set of zonal and meridional velocities on a slightly larger grid. In this case, the meridional velocity is also 1 m/s.

In [ ]:
polygon = np.array([(0, -5), (35, 0), (35, 25), (0, 20)])
grid_polygons.append(polygon)

ds_in.append(setup_nested_ds(polygon))
ds_in[-1]["U"][:] = 1.0
ds_in[-1]["V"][:] = 1.0

Finally, we define another set of velocities on an even larger grid. The zonal velocity is the same as for the smaller grids, but the meridional velocity is now a cosine as a function of longitude.

In [ ]:
polygon = np.array([(-10, -20), (60, -20), (60, 40), (-10, 40)])
grid_polygons.append(polygon)

ds_in.append(setup_nested_ds(polygon))
lon_g, lat_g = np.meshgrid(ds_in[-1]["lon"], ds_in[-1]["lat"])
ds_in[-1]["U"][:] = 1.0
ds_in[-1]["V"][:] = np.cos(lon_g / 5 * np.pi / 2)

We plot the velocity fields on top of each other, indicating the grid boundaries in red.

In [ ]:
n_grids = len(ds_in)
cmap = ListedColormap([f"C{i}" for i in range(n_grids)])

fig, ax = plt.subplots()

for i in range(n_grids):
    ds = ds_in[i].isel(time=0, depth=0)
    ax.quiver(ds.lon, ds.lat, ds.U, ds.V, color=f"C{i}")
    poly = grid_polygons[i]
    ax.plot(
        np.append(poly[:, 0], poly[0, 0]),
        np.append(poly[:, 1], poly[0, 1]),
        "-r",
        lw=1,
    )

plt.tight_layout()
plt.show()

Note, as seen in the plot above, that the dataset domains in this case are rectangular, but the polygons that will later define the nested Grid boundaries don't have to be. So we can even use this method to subset parts of a Grid.

## Creating a Delaunay triangulation of the nested Grids

Now comes the important part: we need to create a Delaunay triangulation of the  nested Grids, so that we can efficiently determine in which Grid a particle is located at any given time. We use the `triangle` package to perform the triangulation, and `shapely` to handle the geometric operations. 

Note that we need to make sure that all the edges of the polygons are also edges of the triangulation. We do this by using a [constrained (PSLG) Delaunay triangulation](https://en.wikipedia.org/wiki/Constrained_Delaunay_triangulation).

The result is a set of triangles covering the nested Grids, which we can use to determine in which Grid a particle is located at any given time. It is important that the list of polygons is ordered from smallest to largest Grid, so that triangles in overlapping areas are assigned to the correct Grid.

In [ ]:
import numpy as np
from shapely.geometry import Point, Polygon
from triangle import triangulate


def constrained_triangulate_keep_edges(polygons):
    """Constrained triangulation while keeping polygon edges.

    Args:
      polygons: list of (Ni,2) numpy arrays (polygon boundary points in order)

    Returns:
      pts (P,2) array of vertex coordinates,
      tris (M,3) array of triangle vertex indices,
      face_poly (M,) mapping each triangle to the polygon index or -1.
    """
    # build vertices + segments for Triangle PSLG
    verts = []
    segments = []
    offset = 0
    for poly in polygons:
        Ni = len(poly)
        verts.extend(poly.tolist())
        segments.extend([[offset + j, offset + ((j + 1) % Ni)] for j in range(Ni)])
        offset += Ni
    verts = np.asarray(verts, dtype=float)
    segments = np.asarray(segments, dtype=int)

    mode = "p"  # "p" = PSLG (constrained triangulation)
    B = triangulate({"vertices": verts, "segments": segments}, mode)

    pts = B["vertices"]
    tris = B["triangles"].astype(int)

    # assign triangles to polygons using centroid test
    shapely_polys = [Polygon(p) for p in polygons]
    centers = pts[tris].mean(axis=1)
    face_poly = np.full(len(tris), -1, dtype=int)
    for ti, c in enumerate(centers):
        for ip in range(len(shapely_polys)):
            if shapely_polys[ip].contains(Point(c)):
                face_poly[ti] = ip
                break

    return pts, tris, face_poly


points, face_tris, face_poly = constrained_triangulate_keep_edges(grid_polygons)

We can then plot the resulting triangles to verify that they correctly cover the nested Grids.

In [ ]:
fig, ax = plt.subplots()
for i in range(n_grids)[::-1]:
    tris = face_tris[face_poly == i]
    ax.triplot(points[:, 0], points[:, 1], tris, label=f"Grid {i}", color=f"C{i}")
ax.scatter(points[:, 0], points[:, 1], s=10, c="k")
ax.set_aspect("equal")
ax.legend()

# reverse legend labels
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1])
plt.show()

Then, we convert the triangulation into an (unstructured) `parcels.FieldSet`.

In [ ]:
# build an xarray dataset compatible with UGRID / uxarray
n_node = points.shape[0]
n_face = face_tris.shape[0]
n_max_face_nodes = face_tris.shape[1]

ds_tri = xr.Dataset(
    {
        "node_lon": (("n_node",), points[:, 0]),
        "node_lat": (("n_node",), points[:, 1]),
        "face_node_connectivity": (("n_face", "n_max_face_nodes"), face_tris),
        "face_polygon": (
            (
                "time",
                "nz",
                "n_face",
            ),
            face_poly[np.newaxis, np.newaxis, :],
            {
                "long_name": "Grid ID",
                "location": "face",
                "mesh": "delaunay",
            },
        ),
    },
    coords={
        "time": np.array([np.timedelta64(0, "ns")]),
        "nz": np.array([0]),
        "n_node": np.arange(n_node),
        "n_face": np.arange(n_face),
    },
    attrs={"Conventions": "UGRID-1.0"},
)

uxda = ux.UxDataArray(ds_tri["face_polygon"], uxgrid=ux.Grid(ds_tri))

GridID = parcels.Field(
    "GridID",
    uxda,
    # TODO note that here we need to use mesh="flat" otherwise the hashing doesn't work. See https://github.com/Parcels-code/Parcels/pull/2439#discussion_r2627664010
    parcels.UxGrid(uxda.uxgrid, z=uxda["nz"], mesh="flat"),
    interp_method=parcels.interpolators.UxPiecewiseConstantFace,
)
fieldset = parcels.FieldSet([GridID])

We can confirm that the FieldSet has been created correctly by running a Parcels simulation where particles sample the `GridID` field, which indicates in which Grid each particle is located at any given time.

In [ ]:
X, Y = np.meshgrid(
    np.linspace(-8, 58, 25),
    np.linspace(-18, 38, 20),
)

NestedGridParticle = parcels.Particle.add_variable(
    parcels.Variable("gridID", dtype=np.int32)
)
pset = parcels.ParticleSet(
    fieldset, pclass=NestedGridParticle, lon=X.flatten(), lat=Y.flatten()
)


def SampleGridID(particles, fieldset):
    particles.gridID = fieldset.GridID[particles]


pset.execute(
    SampleGridID,
    runtime=np.timedelta64(1, "s"),
    dt=np.timedelta64(1, "s"),
    verbose_progress=False,
)

Indeed, the visualisation below shows that particles correctly identify the grid they are in based on their location.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))
plot_args = {"cmap": cmap, "edgecolors": "k", "linewidth": 0.5}

triang = mtri.Triangulation(
    uxda.uxgrid.node_lon.values,
    uxda.uxgrid.node_lat.values,
    triangles=uxda.uxgrid.face_node_connectivity.values,
)
facecolors = np.squeeze(uxda[0, :].values)

ax.tripcolor(triang, facecolors=facecolors, shading="flat", **plot_args)
ax.scatter(pset.lon, pset.lat, c=pset.gridID, **plot_args)
ax.set_aspect("equal")
ax.set_title(
    "Nested Grids visualisation (triangulation and interpolated particle values)"
)
plt.tight_layout()
plt.show()

## Advecting particles with nested Grid transitions

We can now set up a particle advection simulation using the nested Grids. We first combine all the Fields into a single FieldSet. 

We rename the individual Fields by appending the Grid index to their names, so that we can easily identify which Field belongs to which Grid. We also add the `GridID` Field to the FieldSet (note that Parcels v4 supports combining structured and unstructured Fields into one FieldSet, which is very convenient for this usecase).

In [ ]:
fields = [GridID]
for i, ds in enumerate(ds_in):
    # TODO : use FieldSet.from_sgrid_convetion here once #2437 is merged
    grid = parcels.XGrid.from_dataset(ds, mesh="spherical")
    U = parcels.Field("U", ds["U"], grid, interp_method=parcels.interpolators.XLinear)
    V = parcels.Field("V", ds["V"], grid, interp_method=parcels.interpolators.XLinear)
    UV = parcels.VectorField("UV", U, V)
    fset = parcels.FieldSet([U, V, UV])

    for fld in fset.fields.values():
        fld.name = f"{fld.name}{i}"
        fields.append(fld)
fieldset = parcels.FieldSet(fields)

We then define a custom Advection kernel that advects particles using the appropriate velocity Field based on the `GridID` at the particle's location. Note that for simplicity, we use Eulerian advection here, but in a real-world application you would typically use a higher-order scheme.

In [ ]:
def AdvectEE_NestedGrids(particles, fieldset):
    particles.gridID = fieldset.GridID[particles]

    # TODO because of KernelParticle bug (GH #2143), we need to copy lon/lat/time to local variables
    time = particles.time
    z = particles.z
    lat = particles.lat
    lon = particles.lon
    u = np.zeros_like(particles.lon)
    v = np.zeros_like(particles.lat)

    unique_ids = np.unique(particles.gridID)
    for gid in unique_ids:
        mask = particles.gridID == gid
        UVField = getattr(fieldset, f"UV{gid}")
        (u[mask], v[mask]) = UVField[time[mask], z[mask], lat[mask], lon[mask]]

    particles.dlon += u * particles.dt
    particles.dlat += v * particles.dt

    # TODO particle states have to be updated manually because UVField is not called with `particles` argument (becaise of GH #2143)
    particles.state = np.where(
        np.isnan(u) | np.isnan(v),
        parcels.StatusCode.ErrorInterpolation,
        particles.state,
    )


lat = np.linspace(-17, 35, 10)
lon = np.full(len(lat), -5)

pset = parcels.ParticleSet(fieldset, pclass=NestedGridParticle, lon=lon, lat=lat)
ofile = parcels.ParticleFile(
    "nestedgrid_particles.zarr", outputdt=np.timedelta64(1, "D")
)
pset.execute(
    AdvectEE_NestedGrids,
    runtime=np.timedelta64(60, "D"),
    dt=np.timedelta64(1, "D"),
    output_file=ofile,
    verbose_progress=False,
)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))

ds_out = xr.open_zarr("nestedgrid_particles.zarr")

plt.plot(ds_out.lon.T, ds_out.lat.T, "k", linewidth=0.5)
sc = ax.scatter(ds_out.lon, ds_out.lat, c=ds_out.gridID, s=4, cmap=cmap, vmin=0, vmax=2)
xl, yl = ax.get_xlim(), ax.get_ylim()

for i in range(n_grids - 1):
    poly = grid_polygons[i]
    ax.plot(
        np.append(poly[:, 0], poly[0, 0]),
        np.append(poly[:, 1], poly[0, 1]),
        "-r",
        lw=1,
    )
ax.set_xlim(xl)
ax.set_ylim(yl)
ax.set_aspect("equal")

cbar = plt.colorbar(sc, ticks=[0, 1, 2], ax=ax)
cbar.set_label("Grid ID")
ax.set_title("Particle advection through nested Grids")
plt.tight_layout
plt.show()

Indeed, we can see that the particles follow the cosine oscillation pattern in the coarsest grid, move northeast in the finer grid, and move purely zonally in the finest grid.

Note that in the plot above the particles at higher latitudes move farther eastward because of the curvature of the earth, so that is expected.